In [1]:
# %matplotlib inline
# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.random.seed(0)

In [2]:
data = pd.read_csv("/home/irek/PycharmProjects/zprp-ner-active_learning/data/processed/ner_dataset.csv")
data = data.fillna(method="ffill")
data.head(20)

/tmp/ipykernel_4319/4220236873.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [3]:
print("Unique words in corpus:", data["Word"].nunique())
print("Unique tags in corpus:", data["Tag"].nunique())

Unique words in corpus: 35171
Unique tags in corpus: 17


In [4]:
words = ["<PAD>"]
words.extend(list(set(data["Word"].values)))
num_words = len(words)

tags = list(set(data["Tag"].values))
num_tags = len(tags)

In [5]:
def sentence_integrate(data):
    def agg_func(s):
        return [
            (w, p, t)
            for w, p, t in zip(
                s["Word"].values.tolist(),
                s["POS"].values.tolist(),
                s["Tag"].values.tolist(),
            )
        ]

    return data.groupby("Sentence #").apply(agg_func).tolist()


sentences = sentence_integrate(data)

/tmp/ipykernel_4319/538150070.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return data.groupby("Sentence #").apply(agg_func).tolist()


In [8]:
import plotly.express as px

fig = px.histogram(
    pd.DataFrame([len(s) for s in sentences], columns=["length"]),
    x="length",
    marginal="box",
)
fig.show()

In [6]:
sentences[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [7]:
import json

# word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
with open("/home/irek/PycharmProjects/zprp-ner-active_learning/dupa/words_to_idx.json") as f:
    word2idx = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/home/irek/PycharmProjects/zprp-ner-active_learning/dupa/words_to_idx.json'

In [8]:
tag2idx

{'O': 0,
 'B-per': 1,
 'B-org': 2,
 'I-gpe': 3,
 'I-eve': 4,
 'I-nat': 5,
 'B-geo': 6,
 'I-org': 7,
 'I-per': 8,
 'B-eve': 9,
 'I-tim': 10,
 'I-geo': 11,
 'B-art': 12,
 'B-gpe': 13,
 'B-tim': 14,
 'I-art': 15,
 'B-nat': 16}

In [9]:
import numpy as np

import hashlib


def hash_string(s: str) -> int:
    return int(hashlib.sha256(s.encode()).hexdigest(), 16) % 100_001


max_len = 50
X = [[hash_string(w[0]) for w in s] for s in sentences]
Y = [[tag2idx[w[2]] for w in s] for s in sentences]

X = np.array([x[:max_len] + [num_words - 1] * (max_len - len(x)) for x in X])
Y = np.array([y[:max_len] + [tag2idx["O"]] * (max_len - len(y)) for y in Y])
X.shape, Y.shape

((47959, 50), (47959, 50))

35172

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [11]:
import torch.nn as nn
import torch.nn.functional as F


class BiLSTM(nn.Module):
    def __init__(
            self,
            num_words,
            num_classes=17,
            embedding_dim=100,
            lstm_units=100,
            dropout=0.1,
    ):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(100_000, embedding_dim)
        self.bn = nn.BatchNorm1d(embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(
            embedding_dim,
            lstm_units,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
        )
        self.linear = nn.Linear(lstm_units * 2, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.dropout(embedded)
        embedded = self.bn(embedded.permute(0, 2, 1)).permute(0, 2, 1)
        lstm_output, _ = self.lstm(embedded)
        logits = self.linear(lstm_output)
        probabilities = F.softmax(logits, dim=2)
        return probabilities

In [12]:
import torch

torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [13]:
import torch

tag_counts = data["Tag"].value_counts()
print(tag_counts)
# policz sumę wszystkich wystąpień
total_count = tag_counts.sum()

# policz wagi dla każdego tagu
tag_weights = total_count / tag_counts
tag_weights = list(tag_weights.items())
tag_weights = sorted(tag_weights, key=lambda x: tag2idx[x[0]])
tag_weights = torch.tensor([w[1] for w in tag_weights])
tag_weights

Tag
O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: count, dtype: int64


tensor([1.1810e+00, 6.1717e+01, 5.2057e+01, 5.2958e+03, 4.1446e+03, 2.0560e+04,
        2.7855e+01, 6.2475e+01, 6.0783e+01, 3.4045e+03, 1.6063e+02, 1.4143e+02,
        2.6084e+03, 6.6073e+01, 5.1570e+01, 3.5306e+03, 5.2168e+03])

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def eval(model, eval_loader, loss_func, verbose=True, return_values=False):
    running_loss = 0.0
    predicted_labels = []
    true_labels = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in eval_loader:
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(2))
            labels = labels.view(-1)
            loss = loss_func(outputs, labels)
            running_loss += loss.item()
            _, predicted_test = torch.max(outputs, 1)
            predicted_labels.extend(predicted_test.tolist())
            true_labels.extend(labels.tolist())
    loss = running_loss / len(eval_loader)
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average="macro")
    recall = recall_score(true_labels, predicted_labels, average="macro")
    f1 = f1_score(true_labels, predicted_labels, average="macro")
    if verbose:
        print(f"Loss: {loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        print("----------------------------------")
    if return_values:
        return {
            "loss": loss,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }

In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings


# Funkcja do dzielenia macierzy na listę mniejszych macierzy
def split_array(array, chunk_size):
    return [array[i:i + chunk_size] for i in range(0, len(array), chunk_size)]


# Dzielimy x_train i y_train
chunk_size = 32
x_train_chunks = split_array(x_train, chunk_size)
y_train_chunks = split_array(y_train, chunk_size)

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_test_tensor = torch.LongTensor(x_test).to(device)
y_test_tensor = torch.LongTensor(y_test).to(device)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32)

current_x_train = []
current_y_train = []
model = BiLSTM(num_words).to(device)
criterion = nn.CrossEntropyLoss(weight=tag_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)

for idx, (x_chunk, y_chunk) in enumerate(zip(x_train_chunks, y_train_chunks)):
    current_x_train.extend(x_chunk), current_y_train.extend(y_chunk)

    x_train_tensor = torch.LongTensor(current_x_train).to(device)
    y_train_tensor = torch.LongTensor(current_y_train).to(device)

    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    epochs = 20
    print(f"------Chunk {idx + 1}/{len(x_train_chunks)}--------")
    for epoch in range(epochs):
        model.train()

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            # Spłaszczenie do dwóch wymiarów
            outputs = outputs.view(-1, outputs.size(2))
            labels = labels.view(-1)  # Spłaszczenie do jednego wymiaru

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    eval(model, test_loader, criterion, True, False)



------Chunk 1/1199--------
Loss: 2.7481
Accuracy: 0.5712
Precision: 0.1263
Recall: 0.1148
F1-score: 0.0677
----------------------------------
------Chunk 2/1199--------
Loss: 2.6961
Accuracy: 0.5877
Precision: 0.0817
Recall: 0.1688
F1-score: 0.0775
----------------------------------
------Chunk 3/1199--------
Loss: 2.6497
Accuracy: 0.6089
Precision: 0.0871
Recall: 0.2038
F1-score: 0.0905
----------------------------------
------Chunk 4/1199--------
Loss: 2.6319
Accuracy: 0.6280
Precision: 0.0836
Recall: 0.2147
F1-score: 0.0892
----------------------------------
------Chunk 5/1199--------
Loss: 2.6235
Accuracy: 0.8742
Precision: 0.1626
Recall: 0.1989
F1-score: 0.1688
----------------------------------
------Chunk 6/1199--------
Loss: 2.6107
Accuracy: 0.9208
Precision: 0.2220
Recall: 0.2048
F1-score: 0.2054
----------------------------------
------Chunk 7/1199--------
Loss: 2.6016
Accuracy: 0.9354
Precision: 0.2634
Recall: 0.2125
F1-score: 0.2283
----------------------------------
------

KeyboardInterrupt: 

In [16]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_train_tensor = torch.LongTensor(x_train).to(device)
y_train_tensor = torch.LongTensor(y_train).to(device)
x_test_tensor = torch.LongTensor(x_test).to(device)
y_test_tensor = torch.LongTensor(y_test).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32)

model = BiLSTM(num_words).to(device)

criterion = nn.CrossEntropyLoss(weight=tag_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 20

for epoch in range(epochs):
    model.train()

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        # Spłaszczenie do dwóch wymiarów
        outputs = outputs.view(-1, outputs.size(2))
        labels = labels.view(-1)  # Spłaszczenie do jednego wymiaru

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}----------------------")
    print("Train dataset:")
    eval(model, train_loader, criterion, True, False)
    print("Eval dataset:")
    eval(model, test_loader, criterion, True, False)

Epoch 1/20----------------------
Train dataset:
Loss: 2.2426
Accuracy: 0.8647
Precision: 0.2458
Recall: 0.5754
F1-score: 0.3141
----------------------------------
Eval dataset:
Loss: 2.2952
Accuracy: 0.8649
Precision: 0.2386
Recall: 0.5583
F1-score: 0.3032
----------------------------------
Epoch 2/20----------------------
Train dataset:
Loss: 2.1879
Accuracy: 0.9062
Precision: 0.3455
Recall: 0.6293
F1-score: 0.3701
----------------------------------
Eval dataset:
Loss: 2.2498
Accuracy: 0.9040
Precision: 0.2759
Recall: 0.5999
F1-score: 0.3559
----------------------------------
Epoch 3/20----------------------
Train dataset:
Loss: 2.1521
Accuracy: 0.9340
Precision: 0.3574
Recall: 0.6726
F1-score: 0.4376
----------------------------------
Eval dataset:
Loss: 2.2275
Accuracy: 0.9309
Precision: 0.3400
Recall: 0.6217
F1-score: 0.4152
----------------------------------
Epoch 4/20----------------------
Train dataset:
Loss: 2.1238
Accuracy: 0.9454
Precision: 0.4410
Recall: 0.7086
F1-score: 0.4

In [17]:
# i = np.random.randint(0, x_test.shape[0])
i = 0
sentence = [w[0] for w in sentences[i]]
print(sentence)

true_labels = [w[2] for w in sentences[i]]
print(true_labels)

hashed_sentence = [hash_string(w) for w in sentence]
print(hashed_sentence)

s = torch.tensor(
    [
        hashed_sentence,
    ]
).to(device)
print(s)

model.eval()

with torch.no_grad():
    p = np.argmax(model(s).cpu(), axis=-1)

print("{:15}{:5}\t {}\n".format("Word", "True", "Pred"))

predictions = p[0]
for idx, word in enumerate(sentence):
    print("{:15}{}\t{}".format(word, true_labels[idx], tags[predictions[idx]]))


# for w, true, pred in zip(x_test[i], y_test[i], p[0]):
#     # if words[w] == "ENDPAD":
#     #     break
#     print("{:15}{}\t{}".format(words[w], tags[true], tags[pred]))

['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']
[96844, 19376, 40914, 5097, 89921, 94304, 79733, 51786, 96443, 18221, 76265, 91958, 7399, 78694, 12524, 18221, 10937, 19376, 43879, 63804, 86782, 7986, 81819, 61285]
tensor([[96844, 19376, 40914,  5097, 89921, 94304, 79733, 51786, 96443, 18221,
         76265, 91958,  7399, 78694, 12524, 18221, 10937, 19376, 43879, 63804,
         86782,  7986, 81819, 61285]], device='cuda:0')
Word           True 	 Pred

Thousands      O	O
of             O	O
demonstrators  O	O
have           O	O
marched        O	O
through        O	O
London         B-geo	B-geo
to             O	O
protest        O	O
the            O	O
war            O	O
in             O	O
Iraq

In [18]:
model_state = {
    "model_state_dict": model.state_dict()
}

In [19]:
torch.save(model_state, '/home/irek/PycharmProjects/zprp-ner-active_learning/new_model.pth')